# Text Generation using LSTM Network
#### ディープラーニングによる文章生成

## What is LSTM?
  
テキスト生成は一種の言語モデリング問題。  
言語モデリングは、テキスト読み上げ、会話システム、テキスト要約など、多数の自然言語処理タスクの中心的な問題。    
訓練された言語モデルは、テキストで使用されている前の一連の単語に基づいて、単語の出現の可能性を学習する。  
言語モデルは、文字レベル、n-gramレベル、文レベル、または段落レベルでも操作できる。  
このノートでは、最先端のリカレントニューラルネットワークを実装してトレーニングすることによって、自然言語テキストを生成するための言語モデルを作成する方法について説明していく
。

## 今回はニュースの本文からタイトルを自動生成します  
  
#### Process  
1. データの準備  
2. 文章のお掃除（記号削除、小文字統一）  
3. 単語に切り分ける  
4. トークン化＝数値化  
5. パディングで変数の長さを統一  
6. LSTMの実装

## 1. Import the libraries ライブラリのインポート

In [2]:
import pandas as pd
import numpy as np

## 2. Load the dataset データのロード

In [23]:
import os
path = os.getcwd()
print(path)

C:\MyWorks\PythonWorkspace\seq2seqHome\Text_Generation_using_GRU


In [4]:
ls

 Volume in drive C is Windows
 Volume Serial Number is E050-A3EE

 Directory of C:\MyWorks\PythonWorkspace\seq2seqHome\Text_Generation_using_GRU

2020-02-07  09:31 PM    <DIR>          .
2020-02-07  09:31 PM    <DIR>          ..
2020-02-07  09:23 PM    <DIR>          .ipynb_checkpoints
2020-02-07  09:31 PM            16,100 0315_Text_Generation_with_GRU.ipynb
2020-02-07  09:18 PM               742 attention.py
2020-02-07  09:18 PM             4,632 lstm.py
2020-02-07  09:18 PM             4,626 my_lstm.py
2020-02-07  09:24 PM               555 Neural-Text-Generation.ipynb
2019-10-01  06:05 PM        83,917,554 News_Category_Dataset_v2.json
2020-02-07  09:25 PM        26,677,036 News_Category_Dataset_v2.json.zip
2020-02-07  09:18 PM             2,718 README.md
2020-02-07  09:18 PM            23,413 RECENT_Notenool.ipynb
2020-02-07  09:18 PM               744 seq2seq_attention.py
2020-02-07  09:18 PM            14,238 slack_model.py
2020-02-07  09:22 PM           179,696 Text_Generation_

# import pandas as pd
import json
my_json_file=path+"/news-category-small.json"
data = [json.loads(line) for line in open(my_json_file, 'r')]
print(data[1])



## reading the headlines
import json  
from pandas.io.json import json_normalize  

#with open(path +"/News_Category_Dataset_v2.json") as f: 
# d = json.loads(f) 
df=pd.read_json(path +"/News_Category_Dataset_v2.json")          
print(df)
#headlines= json_normalize(d["headline"].value()) 
#print(headlines.head())




In [38]:
import json 

my_json_file=path+"/news-category-small.json" 

data = [json.loads(line) for line in open(my_json_file, 'r')] 

#for row in data:
#    print(row["headline"])

headlines=[row["headline"]  for row in data]

print(len(headlines))
print(headlines)


30
['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 'Hugh Grant Marries For The First Time At Age 57', "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork", 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog', "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy", "Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit", 'What To Watch On Amazon Prime That’s New This Week', "Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film", 'What To Watch On Hulu That’s New This Week', 'Justin Timberlake Visits Texas School Shooting Victims', "South Korean President Meets North Korea's Kim Jong Un To Talk Trump Summit", 'With Its Way Of Life At Risk, This Remote Oyster-Growing Region Called In Robots', "Trump's Crackdown On Immigrant Parents Puts More Kids In An Already Strained System", "'Trump's Son Should

In [18]:
headlines = []
for filename in os.listdir(path):
    if "Articles" in filename:
        article_df = pd.read_csv(path + "/New York Times/" + filename)
        headlines.extend(list(article_df["headline"].values))
        break
        
headlines = [ h for h in headlines if h != "Unknown" ]
print("The number of headline is :", len(headlines))

The number of headline is : 0


In [16]:
headlines

[]

## 3. Dataset preparation 前処理

### 3.1 Dataset cleaning  データクリーニング
  
記号を取り除き、「文字」と「数字」だけ残す。また小文字で統一する。

In [39]:
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

# 元データのタイトルに含まれる単語群から独自のコーパスを作成
corpus = [ clean_text(headline) for headline in headlines ]

In [40]:
corpus[:5]

['there were 2 mass shootings in texas last week but only 1 on tv',
 'will smith joins diplo and nicky jam for the 2018 world cups official song',
 'hugh grant marries for the first time at age 57',
 'jim carrey blasts castrato adam schiff and democrats in new artwork',
 'julianna margulies uses donald trump poop bags to pick up after her dog']

### 3.2 Generating Sequence of N-gram Tokens 文章の単語化&数値化

- 自然言語処理では、テキストを単語単位に分解してベクトル化するのが主流である。  
- N-gram は、Morphological Analysis（形態要素解析）に並ぶ代表的な単語の切り出し手法のひとつ。  
- 具体的には、N-gramとは自然言語（テキスト）を連続するN個の文字、もしくはN個の単語単位で切り出す手法のこと。  
- 強みは「コーパス」が事前に入らないこと、弱みは切り出した単語数が肥大化しやすい点。  
- ex:) "I voted for Trump." n=2 => "I voted", "for Trump"

In [59]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocabraly = set(vocab)

In [60]:
len(vocabraly)

265

In [43]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(2000)
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

Using TensorFlow backend.


265

In [44]:
word2index

{'to': 1,
 'in': 2,
 'on': 3,
 'trump': 4,
 'new': 5,
 'trumps': 6,
 'week': 7,
 'and': 8,
 'the': 9,
 'at': 10,
 'this': 11,
 'with': 12,
 'abortion': 13,
 '2': 14,
 'texas': 15,
 'but': 16,
 'for': 17,
 'first': 18,
 'donald': 19,
 'after': 20,
 'what': 21,
 'watch': 22,
 'prime': 23,
 'thats': 24,
 'north': 25,
 'summit': 26,
 'of': 27,
 'more': 28,
 'putin': 29,
 'than': 30,
 'ireland': 31,
 'landslide': 32,
 'men': 33,
 'he': 34,
 'there': 35,
 'were': 36,
 'mass': 37,
 'shootings': 38,
 'last': 39,
 'only': 40,
 '1': 41,
 'tv': 42,
 'will': 43,
 'smith': 44,
 'joins': 45,
 'diplo': 46,
 'nicky': 47,
 'jam': 48,
 '2018': 49,
 'world': 50,
 'cups': 51,
 'official': 52,
 'song': 53,
 'hugh': 54,
 'grant': 55,
 'marries': 56,
 'time': 57,
 'age': 58,
 '57': 59,
 'jim': 60,
 'carrey': 61,
 'blasts': 62,
 'castrato': 63,
 'adam': 64,
 'schiff': 65,
 'democrats': 66,
 'artwork': 67,
 'julianna': 68,
 'margulies': 69,
 'uses': 70,
 'poop': 71,
 'bags': 72,
 'pick': 73,
 'up': 74,
 'her':

In [45]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    if idx > 1406:
        continue
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [46]:
max(rev_dictionary.keys())

265

In [47]:
input_seqences = tokenizer.texts_to_sequences(corpus)

In [48]:
input_seqences[:5]

[[35, 36, 14, 37, 38, 2, 15, 39, 7, 16, 40, 41, 3, 42],
 [43, 44, 45, 46, 8, 47, 48, 17, 9, 49, 50, 51, 52, 53],
 [54, 55, 56, 17, 9, 18, 57, 10, 58, 59],
 [60, 61, 62, 63, 64, 65, 8, 66, 2, 5, 67],
 [68, 69, 70, 19, 4, 71, 72, 1, 73, 74, 20, 75, 76]]

In [49]:
len(input_seqences)

30

### 3.3 Padding the Sequences and obtain Variables
#### パディングによって固定長データを作り、説明変数を得る

In [61]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])

In [62]:
input_data[:5]

[[35], [35, 36], [35, 36, 14], [35, 36, 14, 37], [35, 36, 14, 37, 38]]

In [63]:
target[:5]

[14, 37, 38, 2, 15]

In [64]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

14

In [73]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")

In [74]:
len(input_data[0])

14

In [67]:
input_data

array([[ 35,   0,   0, ...,   0,   0,   0],
       [ 35,  36,   0, ...,   0,   0,   0],
       [ 35,  36,  14, ...,   0,   0,   0],
       ...,
       [255, 256,  10, ...,   0,   0,   0],
       [255, 256,  10, ...,   0,   0,   0],
       [255, 256,  10, ...,   0,   0,   0]])

In [58]:
input_data.shape

(275, 14)

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 37,
 38,
 39,
 40,
 41,
 42,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 56,
 57,
 58,
 59,
 62,
 63,
 64,
 65,
 66,
 67,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 109,
 110,
 111,
 112,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 153,
 154,
 155,
 156,
 157,
 160,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 199,
 200,
 201,
 202,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 

In [80]:
from keras.utils import to_categorical
total_words = 266
target = to_categorical(target, num_classes=total_words)

In [81]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [82]:
target.shape

(275, 266)

In [84]:
VOCAB_SIZE = 2001
VOCAB_SIZE

2001

In [86]:
MAX_LEN

14

## 4. LSTMs for Text Generation 長短期記憶層アルゴリズムの実装

### 4.1 LSTM ( Long Short-Term Memory  )    
  
1. Input Layer : Takes the sequence of words as input  
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.  
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer.  
4. Output Layer : Computes the probability of the best possible next word as output  

![title](https://cdn-images-1.medium.com/max/1600/1*yBXV9o5q7L_CvY7quJt3WQ.png)

In [89]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping

In [90]:
model = Sequential()

"""入力層"""
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))

"""隠れ層"""
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))

"""出力層 活性化関数は多層のソフトマックス関数"""
model.add(Dense(units=target.shape[1], activation="softmax"))

In [91]:
"""損失関数と最適化手法の設定"""
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [92]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 14, 100)           200100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 266)               26866     
Total params: 307,366
Trainable params: 307,366
Non-trainable params: 0
_________________________________________________________________


#### LSTMモデルにテストデータを学習させていく！

In [95]:
"""予測ラベルと正解ラベルを用意する"""
model.fit(input_data, target, batch_size=32, epochs=500, verbose=1)

Epoch 1/500
275/275 [==============================] - 0s 508us/step - loss: 0.6727
Epoch 2/500
275/275 [==============================] - 0s 432us/step - loss: 0.6406
Epoch 3/500
275/275 [==============================] - 0s 392us/step - loss: 0.6239
Epoch 4/500
275/275 [==============================] - 0s 395us/step - loss: 0.6633
Epoch 5/500
275/275 [==============================] - 0s 388us/step - loss: 0.6470
Epoch 6/500
275/275 [==============================] - 0s 403us/step - loss: 0.6148
Epoch 7/500
275/275 [==============================] - 0s 421us/step - loss: 0.6018
Epoch 8/500
275/275 [==============================] - 0s 392us/step - loss: 0.6436
Epoch 9/500
275/275 [==============================] - 0s 424us/step - loss: 0.6165
Epoch 10/500
275/275 [==============================] - 0s 432us/step - loss: 0.6399
Epoch 11/500
275/275 [==============================] - 0s 446us/step - loss: 0.5937
Epoch 12/500
275/275 [==============================] - 0s 421us/step - lo

275/275 [==============================] - 0s 392us/step - loss: 0.2808
Epoch 98/500
275/275 [==============================] - 0s 392us/step - loss: 0.2770
Epoch 99/500
275/275 [==============================] - 0s 388us/step - loss: 0.2560
Epoch 100/500
275/275 [==============================] - 0s 417us/step - loss: 0.2569
Epoch 101/500
275/275 [==============================] - 0s 392us/step - loss: 0.2668
Epoch 102/500
275/275 [==============================] - 0s 381us/step - loss: 0.2476
Epoch 103/500
275/275 [==============================] - 0s 381us/step - loss: 0.2456
Epoch 104/500
275/275 [==============================] - 0s 395us/step - loss: 0.2587
Epoch 105/500
275/275 [==============================] - 0s 392us/step - loss: 0.2607
Epoch 106/500
275/275 [==============================] - 0s 381us/step - loss: 0.2712
Epoch 107/500
275/275 [==============================] - 0s 381us/step - loss: 0.2498
Epoch 108/500
275/275 [==============================] - 0s 388us/step

275/275 [==============================] - 0s 388us/step - loss: 0.1938
Epoch 193/500
275/275 [==============================] - 0s 381us/step - loss: 0.2246
Epoch 194/500
275/275 [==============================] - 0s 381us/step - loss: 0.3053
Epoch 195/500
275/275 [==============================] - 0s 384us/step - loss: 0.4471
Epoch 196/500
275/275 [==============================] - 0s 399us/step - loss: 0.5223
Epoch 197/500
275/275 [==============================] - 0s 384us/step - loss: 0.4772
Epoch 198/500
275/275 [==============================] - 0s 384us/step - loss: 0.4835
Epoch 199/500
275/275 [==============================] - 0s 388us/step - loss: 0.6352
Epoch 200/500
275/275 [==============================] - 0s 381us/step - loss: 0.5371
Epoch 201/500
275/275 [==============================] - 0s 410us/step - loss: 0.6482
Epoch 202/500
275/275 [==============================] - 0s 395us/step - loss: 0.7560
Epoch 203/500
275/275 [==============================] - 0s 384us/st

275/275 [==============================] - 0s 384us/step - loss: 0.0934
Epoch 288/500
275/275 [==============================] - 0s 392us/step - loss: 0.0953
Epoch 289/500
275/275 [==============================] - 0s 377us/step - loss: 0.0895
Epoch 290/500
275/275 [==============================] - 0s 374us/step - loss: 0.0852
Epoch 291/500
275/275 [==============================] - 0s 388us/step - loss: 0.0882
Epoch 292/500
275/275 [==============================] - 0s 410us/step - loss: 0.0830
Epoch 293/500
275/275 [==============================] - 0s 435us/step - loss: 0.0991
Epoch 294/500
275/275 [==============================] - 0s 432us/step - loss: 0.0968
Epoch 295/500
275/275 [==============================] - 0s 428us/step - loss: 0.0933
Epoch 296/500
275/275 [==============================] - 0s 428us/step - loss: 0.0871
Epoch 297/500
275/275 [==============================] - 0s 435us/step - loss: 0.0838
Epoch 298/500
275/275 [==============================] - 0s 432us/st

275/275 [==============================] - 0s 450us/step - loss: 0.1034
Epoch 383/500
275/275 [==============================] - 0s 450us/step - loss: 0.2057
Epoch 384/500
275/275 [==============================] - 0s 442us/step - loss: 0.4169
Epoch 385/500
275/275 [==============================] - 0s 461us/step - loss: 0.5910
Epoch 386/500
275/275 [==============================] - 0s 482us/step - loss: 0.7110
Epoch 387/500
275/275 [==============================] - 0s 461us/step - loss: 0.7627
Epoch 388/500
275/275 [==============================] - 0s 428us/step - loss: 0.7219
Epoch 389/500
275/275 [==============================] - 0s 413us/step - loss: 0.7810
Epoch 390/500
275/275 [==============================] - 0s 421us/step - loss: 0.7143
Epoch 391/500
275/275 [==============================] - 0s 421us/step - loss: 0.5687
Epoch 392/500
275/275 [==============================] - 0s 435us/step - loss: 0.4975
Epoch 393/500
275/275 [==============================] - 0s 432us/st

275/275 [==============================] - 0s 417us/step - loss: 0.0607
Epoch 478/500
275/275 [==============================] - 0s 406us/step - loss: 0.0535
Epoch 479/500
275/275 [==============================] - 0s 406us/step - loss: 0.0564
Epoch 480/500
275/275 [==============================] - 0s 399us/step - loss: 0.0533
Epoch 481/500
275/275 [==============================] - 0s 410us/step - loss: 0.0502
Epoch 482/500
275/275 [==============================] - 0s 406us/step - loss: 0.0616
Epoch 483/500
275/275 [==============================] - 0s 461us/step - loss: 0.0488
Epoch 484/500
275/275 [==============================] - 0s 442us/step - loss: 0.0550
Epoch 485/500
275/275 [==============================] - 0s 439us/step - loss: 0.0539
Epoch 486/500
275/275 [==============================] - 0s 432us/step - loss: 0.0584
Epoch 487/500
275/275 [==============================] - 0s 410us/step - loss: 0.0526
Epoch 488/500
275/275 [==============================] - 0s 406us/st

### 4.2 GRU ( Gated recurrent unit )

In [96]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
gru_model.add(GRU(units=100))
gru_model.add(Dropout(rate=0.1))
gru_model.add(Dense(units=target.shape[1], activation="softmax"))

In [97]:
gru_model.compile(loss="categorical_crossentropy", optimizer="adam")

In [98]:
gru_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 14, 100)           200100    
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 266)               26866     
Total params: 287,266
Trainable params: 287,266
Non-trainable params: 0
_________________________________________________________________


#### GRUモデルにテストデータを学習させていく！

In [101]:
gru_model.fit(input_data, target, batch_size=32, epochs=400, verbose=1)

Epoch 1/400
275/275 [==============================] - 0s 479us/step - loss: 0.3168
Epoch 2/400
275/275 [==============================] - 0s 406us/step - loss: 0.3119
Epoch 3/400
275/275 [==============================] - 0s 490us/step - loss: 0.3083
Epoch 4/400
275/275 [==============================] - 0s 428us/step - loss: 0.3144
Epoch 5/400
275/275 [==============================] - 0s 453us/step - loss: 0.3100
Epoch 6/400
275/275 [==============================] - 0s 428us/step - loss: 0.3253
Epoch 7/400
275/275 [==============================] - 0s 432us/step - loss: 0.3104
Epoch 8/400
275/275 [==============================] - 0s 410us/step - loss: 0.2999
Epoch 9/400
275/275 [==============================] - 0s 432us/step - loss: 0.2985
Epoch 10/400
275/275 [==============================] - 0s 421us/step - loss: 0.2919
Epoch 11/400
275/275 [==============================] - 0s 435us/step - loss: 0.3033
Epoch 12/400
275/275 [==============================] - 0s 435us/step - lo

275/275 [==============================] - 0s 406us/step - loss: 0.1772
Epoch 98/400
275/275 [==============================] - 0s 424us/step - loss: 0.1604
Epoch 99/400
275/275 [==============================] - 0s 439us/step - loss: 0.1774
Epoch 100/400
275/275 [==============================] - 0s 399us/step - loss: 0.1596
Epoch 101/400
275/275 [==============================] - 0s 399us/step - loss: 0.1606
Epoch 102/400
275/275 [==============================] - 0s 471us/step - loss: 0.1558
Epoch 103/400
275/275 [==============================] - 0s 457us/step - loss: 0.1621
Epoch 104/400
275/275 [==============================] - 0s 432us/step - loss: 0.1459
Epoch 105/400
275/275 [==============================] - 0s 406us/step - loss: 0.1437
Epoch 106/400
275/275 [==============================] - 0s 410us/step - loss: 0.1444
Epoch 107/400
275/275 [==============================] - 0s 413us/step - loss: 0.1557
Epoch 108/400
275/275 [==============================] - 0s 464us/step

275/275 [==============================] - 0s 457us/step - loss: 0.0998
Epoch 193/400
275/275 [==============================] - 0s 479us/step - loss: 0.1137
Epoch 194/400
275/275 [==============================] - 0s 432us/step - loss: 0.1089
Epoch 195/400
275/275 [==============================] - 0s 435us/step - loss: 0.1049
Epoch 196/400
275/275 [==============================] - 0s 424us/step - loss: 0.1044
Epoch 197/400
275/275 [==============================] - 0s 428us/step - loss: 0.1099
Epoch 198/400
275/275 [==============================] - 0s 424us/step - loss: 0.1016
Epoch 199/400
275/275 [==============================] - 0s 432us/step - loss: 0.0994
Epoch 200/400
275/275 [==============================] - 0s 432us/step - loss: 0.1000
Epoch 201/400
275/275 [==============================] - 0s 442us/step - loss: 0.0932
Epoch 202/400
275/275 [==============================] - 0s 486us/step - loss: 0.1052
Epoch 203/400
275/275 [==============================] - 0s 432us/st

275/275 [==============================] - 0s 435us/step - loss: 0.0799
Epoch 288/400
275/275 [==============================] - 0s 439us/step - loss: 0.0687
Epoch 289/400
275/275 [==============================] - 0s 475us/step - loss: 0.0805
Epoch 290/400
275/275 [==============================] - 0s 468us/step - loss: 0.0691
Epoch 291/400
275/275 [==============================] - 0s 486us/step - loss: 0.0731
Epoch 292/400
275/275 [==============================] - 0s 428us/step - loss: 0.0704
Epoch 293/400
275/275 [==============================] - 0s 424us/step - loss: 0.0681
Epoch 294/400
275/275 [==============================] - 0s 421us/step - loss: 0.0733
Epoch 295/400
275/275 [==============================] - 0s 421us/step - loss: 0.0689
Epoch 296/400
275/275 [==============================] - 0s 424us/step - loss: 0.0681
Epoch 297/400
275/275 [==============================] - 0s 432us/step - loss: 0.0776
Epoch 298/400
275/275 [==============================] - 0s 435us/st

275/275 [==============================] - 0s 439us/step - loss: 0.0534
Epoch 383/400
275/275 [==============================] - 0s 464us/step - loss: 0.0553
Epoch 384/400
275/275 [==============================] - 0s 486us/step - loss: 0.0480
Epoch 385/400
275/275 [==============================] - 0s 439us/step - loss: 0.0481
Epoch 386/400
275/275 [==============================] - 0s 435us/step - loss: 0.0438
Epoch 387/400
275/275 [==============================] - 0s 453us/step - loss: 0.0488
Epoch 388/400
275/275 [==============================] - 0s 490us/step - loss: 0.0476
Epoch 389/400
275/275 [==============================] - 0s 479us/step - loss: 0.0479
Epoch 390/400
275/275 [==============================] - 0s 439us/step - loss: 0.0440
Epoch 391/400
275/275 [==============================] - 0s 442us/step - loss: 0.0492
Epoch 392/400
275/275 [==============================] - 0s 471us/step - loss: 0.0491
Epoch 393/400
275/275 [==============================] - 0s 464us/st

## 5. Generating the text タイトルにふさわしいテキストを自動生成する

In [104]:
import tensorflow
import numpy
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

ImportError: cannot import name 'set_random_seed' from 'tensorflow' (c:\users\masudrahman\appdata\local\programs\python\python37\lib\site-packages\tensorflow\__init__.py)

In [117]:
def text_generater(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="post")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [118]:
text1 = "Rudy Giuliani"
text_generater(text1, 5, model, MAX_LEN)

'Rudy Giuliani Way Way Animal Life Reel'

In [119]:
text_generater(text1, 5, gru_model, MAX_LEN)

'Rudy Giuliani Blasts 2 And This To'

## I need more data to training I guess.... 学習不足